In [ ]:
import numpy as np
import pandas as pd

#pre processing 
import re
import string
import nltk.downloader
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

#tree
import random

#serialization
import dill

# Define Text Cleaner


In [ ]:
class TextPreprocessor():
    
    '''
    Preprocess a string for NLP
    
    Inputs
        mode: 'int' 1 for lematization, 0 for stemming
    '''

    def __init__(self, mode: 'int' = 1):

        self.mode = mode

        nltk.download('stopwords')
        self.stop_words = stopwords.words('english')

        if mode:
            
            nltk.download('wordnet')
            nltk.download('averaged_perceptron_tagger')
            self.tokenizer = WordNetLemmatizer()
            
        else:
            
            self.tokenizer = PorterStemmer()
            
    #run the fulling cleaning process
    def prepText(self,text):
    
        if self.mode:
            
            text = self._cleanText(text)
            words = self._tokenize(text)
            words = self._removeStops(words)
            words = self._lematize(words)
            
        else:
            
            text = self._cleanText(text)
            words = self._tokenize(text)
            words = self._removeStops(words)
            words = self._stem(words)
    
        return words
            
    def _cleanText(self,text):

        #cast to lower case
        text = text.lower()
    
        #define regex for links and remove them
        text_dn =re.sub(r'http\S+','',text,)
    
        #remove non letters ,digits, and spaces
        text_ascii = "".join([char for char in text_dn if char in string.ascii_lowercase+' '+string.digits])

        return text_ascii
    
    def _tokenize(self,text):
    
        #splits words on spaces
        words = text.split(" ")

        #drop words that are just spaces
        words = [word for word in words if word != ""]

        return words
    
    def _removeStops(self,words):
    
        #drop all stops words
        return [word for word in words if  word not in self.stop_words]
    
    def _lematize(self,words):
    
        #lematize cleaned words
        return [self.tokenizer.lemmatize(word) for word in words]
  
    def _stem(self, words):
        
        #stem cleaned words
        return [self.tokenizer.stem(word) for word in words]


# Define Random Forest

In [ ]:
class RandomForest():
    
    def __init__(self,X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
        tree_ls = list()
        oob_ls = list()
        for i in range(n_estimators):
            X_bootstrap, y_bootstrap, X_oob, y_oob = self.draw_bootstrap(X_train, y_train)
            tree = self.build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
            tree_ls.append(tree)
            oob_error = self.oob_score(tree, X_oob, y_oob)
            oob_ls.append(oob_error)
        print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
        self.tree_ls = tree_ls  

        
    def entropy(self, p):
        
        if p == 0:
            
            return 0
        
        elif p == 1:
            
            return 0
        
        else:
            return - (p * np.log2(p) + (1 - p) * np.log2(1-p))
        
    def information_gain(self, left_child, right_child):
        
        parent = left_child + right_child
        p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
        p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
        p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
        info_gain_p = self.entropy(p_parent)
        info_gain_l = self.entropy(p_left)
        info_gain_r = self.entropy(p_right)
        return (info_gain_p 
                - len(left_child) / len(parent) * info_gain_l 
                - len(right_child) / len(parent) * info_gain_r)
    
    

    def draw_bootstrap(self, X_train, y_train):
        
        bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
        oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
        self.X_bootstrap = X_train.iloc[bootstrap_indices].values
        self.y_bootstrap = y_train[bootstrap_indices]
        self.X_oob = X_train.iloc[oob_indices].values
        self.y_oob = y_train[oob_indices]
        return self.X_bootstrap, self.y_bootstrap, self.X_oob, self.y_oob
    
    def oob_score(self,tree, X_test, y_test):
        
        mis_label = 0
        for i in range(len(X_test)):
            pred = self.predict_tree(tree, X_test[i])
            if pred != y_test[i]:
                mis_label += 1
        return mis_label / len(X_test)
    
    def find_split_point(self, X_bootstrap, y_bootstrap, max_features):
        feature_ls = list()
        num_features = len(X_bootstrap[0])
        
        while len(feature_ls) <= max_features:
            feature_idx = random.sample(range(num_features), 1)
            if feature_idx not in feature_ls:
                feature_ls.extend(feature_idx)
        
        best_info_gain = -999
        node = None
        for feature_idx in feature_ls:
            for split_point in X_bootstrap[:,feature_idx]:
                left_child = {'X_bootstrap': [], 'y_bootstrap': []}
                right_child = {'X_bootstrap': [], 'y_bootstrap': []}
                
                # split children for continuous variables
                if type(split_point) in [int, float]:
                    for i, value in enumerate(X_bootstrap[:,feature_idx]):
                        if value <= split_point:
                            left_child['X_bootstrap'].append(X_bootstrap[i])
                            left_child['y_bootstrap'].append(y_bootstrap[i])
                        else:
                            right_child['X_bootstrap'].append(X_bootstrap[i])
                            right_child['y_bootstrap'].append(y_bootstrap[i])
                # split children for categoric variables
                else:
                    for i, value in enumerate(X_bootstrap[:,feature_idx]):
                        if value == split_point:
                            left_child['X_bootstrap'].append(X_bootstrap[i])
                            left_child['y_bootstrap'].append(y_bootstrap[i])
                        else:
                            right_child['X_bootstrap'].append(X_bootstrap[i])
                            right_child['y_bootstrap'].append(y_bootstrap[i])
                
                split_info_gain = self.information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
                if split_info_gain > best_info_gain:
                    best_info_gain = split_info_gain
                    left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                    right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                    node = {'information_gain': split_info_gain, 
                            'left_child': left_child, 
                            'right_child': right_child, 
                            'split_point': split_point,
                            'feature_idx': feature_idx}
                    
        return node
    
    def terminal_node(self, node):
        
        y_bootstrap = node['y_bootstrap']
        pred = max(y_bootstrap, key = y_bootstrap.count)
        return pred
    
    def split_node(self, node, max_features, min_samples_split, max_depth, depth):
        left_child = node['left_child']
        right_child = node['right_child']    
    
        del(node['left_child'])
        del(node['right_child'])
        
        if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
            empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
            node['left_split'] = self.terminal_node(empty_child)
            node['right_split'] = self.terminal_node(empty_child)
            return
        
        if depth >= max_depth:
            node['left_split'] = self.terminal_node(left_child)
            node['right_split'] = self.terminal_node(right_child)
            return node
        
        if len(left_child['X_bootstrap']) <= min_samples_split:
            node['left_split'] = node['right_split'] = self.terminal_node(left_child)
        else:
            node['left_split'] = self.find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
            self.split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
        if len(right_child['X_bootstrap']) <= min_samples_split:
            node['right_split'] = node['left_split'] = self.terminal_node(right_child)
        else:
            node['right_split'] = self.find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
            self.split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)
        
    def build_tree(self,X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
        
        root_node = self.find_split_point(X_bootstrap, y_bootstrap, max_features)
        self.split_node(root_node, max_features, min_samples_split, max_depth, 1)
        return root_node
 
    

    def predict_tree(self, tree, X_test):
        feature_idx = tree['feature_idx']
        
        if X_test[feature_idx] <= tree['split_point']:
            if type(tree['left_split']) == dict:
                return self.predict_tree(tree['left_split'], X_test)
            else:
                value = tree['left_split']
                return value
        else:
            if type(tree['right_split']) == dict:
                return self.predict_tree(tree['right_split'], X_test)
            else:
                return tree['right_split']
            
    

    def predict_rf(self, X_test):
        pred_ls = list()
        for i in range(len(X_test)):
            ensemble_preds = [self.predict_tree(tree, X_test.values[i]) for tree in self.tree_ls]
            final_pred = max(ensemble_preds, key = ensemble_preds.count)
            pred_ls.append(final_pred)
        return np.array(pred_ls)

# Define TFIDF

In [ ]:
class TFIDF():
    
    def __init__(self, data = None, column1 = 0, column2 = 0):
        
        self.data = data
        self.column1 = column1
        self.column2 = column2
        self.N = len(self.data[self.column1])
        
    
    #runs the embedding
    def get_tfidf(self):
        
        self._word_frequncey()
        self._get_unique_words()
        self._document_frequencey()
        self._compute_tfidf()
        
        return self.score_matrix
        
    #calculates the word freqeuncey per document   
    def _word_frequncey(self):
        
        self.texts = {}
        
        for word_list ,text in zip(self.data[self.column1],self.data[self.column2]):
            
            word_count = {}
            
            for word in word_list:
                
                if word in word_count:
                    
                    word_count[word] = word_count[word] + 1
                
                else:
                    
                    word_count[word] = 1
                    
            self.texts[text] = word_count
            
        return self.texts
    
    #gets a list of unique words in the DataFrame
    def _get_unique_words(self):
        
        
        self.unique_words  = []
        
        for text in self.texts.values(): 
        
            for key in text.keys():
                
                self.unique_words.append(key)
                
        self.unique_words = set(self.unique_words)
        
        self.M = len(self.unique_words)
        
        return self.unique_words
    
    #calculates the codument frequency 
    def _document_frequencey(self):
        
        self.document_frequencey = {}
        
        for word1 in self.unique_words:
            
            count = 0
        
            for doc in self.data[self.column1]:
            
                
                for word2  in set(doc):
                    
                    if word2 == word1:
                        
                        count = count + 1/self.N
            
            self.document_frequencey[word1] = count
            
        return self.document_frequencey
    
    #computes the embedding matrix
    def _compute_tfidf(self):
        
        self.score_matrix = np.zeros((self.N,self.M))
        
        for i, text in enumerate(self.data[self.column2]):
        
            for j, word in enumerate(self.unique_words):
                
                if word in self.texts[text]:
            
                    self.score_matrix[i,j] = self.texts[text][word] * np.log(1.0/self.document_frequencey[word])
                
        return self.score_matrix

# Import Data and Pre Process

In [ ]:
df = pd.read_csv("Tweet Data.csv")

processor = TextPreprocessor(mode=1)

df = df.reindex(columns = ['message','hashtag','label', 'words'])

df['words'] = df['words'].astype('object')

for i,text in enumerate(df['message']):

  df.at[i, 'words'] = processor.prepText(text)



# Embedd with TFIDF

In [ ]:
embedder = TFIDF(data = df, column1 = 'words', column2 = 'message')

score_matrix = embedder.get_tfidf()

# Train Random Forest

In [ ]:
X_train = pd.DataFrame(score_matrix[:-3000])
y_train = df['label'].to_numpy()[:-3000]

X_val = pd.DataFrame(score_matrix[-3000:-1500])
y_val = df['label'].to_numpy()[-3000:-1500]

X_test = pd.DataFrame(score_matrix[-1500:])
y_test = df['label'].to_numpy()[-1500:]


forest = RandomForest(X_train, y_train, n_estimators= 10, max_features = np.round(score_matrix.shape[1]**0.5), max_depth = 3, min_samples_split = 2)

preds = forest.predict_rf(X_val)

acc = sum(preds == y_val) / len(y_val)
print("Testing accuracy: {}".format(np.round(acc,3)))